<a href="https://colab.research.google.com/github/tchtinku/DALLE-mini/blob/main/DALLE_mini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Installation & Setup

In [2]:
!pip install jax==0.3.25 jaxlib==0.3.25 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html

!pip install -q dalle-mini
!pip install -q git+https://github.com/patil-suraj/vqgan-jax.git

Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.4/197.4 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

###We load required models:
DALL·E mini for text to encoded images
VQGAN for decoding images
CLIP for scoring predictions

In [3]:
DALLE_MODEL = "dalle-mini/dalle-mini/mega-1-fp16:latest"
DALLE_COMMIT_ID = None

VOGAN_REPO =  "dalle-mini/vqgan_imagenet_f16_16384"
VOGAN_COMMIT_ID = "e93a26e7707683d349bf5d5c41c5b0ef69b677a9"

In [13]:
!pip install vqgan-jax

In [14]:
import jax
import jax.numpy as jnp

jax.local_device_count()

1

In [19]:
!pip install dalle-mini==0.6.0
!pip install flax
!pip install git+https://github.com/EleutherAI/vqgan-jax.git
!pip install numpy
!pip install jax
!pip install optax

from dalle_mini import DalleBart, DalleBartProcessor
from vqgan_jax.modelling_flax_vqgan import VQModel
from transformers import FlaxCLIPModel, CLIPProcessor
!pip show vqgan-jax
!pip install vqgan-jax
from vqgan_jax.flax_vqgan import VQModel
from vqgan_jax.flax_vqgan import VQModel

model, params = DalleBart.from_pretrained(
    "dalle-mini/dalle-mini/3b-float16", revision="fp16", dtype=jnp.float16, _do_init=False
)

vqgan, vqgan_params = VQModel.from_pretrained(
    "dalle-mini/vqgan_40M", revision="fp16", _do_init=False
)

ERROR: Could not find a version that satisfies the requirement dalle-mini==0.6.0 (from versions: 0.0.1.dev1, 0.0.2, 0.0.3, 0.0.5, 0.0.6, 0.1.0, 0.1.1, 0.1.2, 0.1.3, 0.1.4, 0.1.5)
ERROR: No matching distribution found for dalle-mini==0.6.0
  Cloning https://github.com/EleutherAI/vqgan-jax.git to /tmp/pip-req-build-ulc1tw5m
  Running command git clone --filter=blob:none --quiet https://github.com/EleutherAI/vqgan-jax.git /tmp/pip-req-build-ulc1tw5m
  fatal: could not read Username for 'https://github.com': No such device or address
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/EleutherAI/vqgan-jax.git /tmp/pip-req-build-ulc1tw5m did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/EleutherAI/vqgan-jax.git /tmp/pip-req-build-ulc1tw5m

ModuleNotFoundError: No module named 'vqgan_jax.modelling_flax_vqgan'

Model parameters are replicated on each device for faster inference.

In [18]:
from flax.jax_utils import replicate

params = replicate(params)
vqgan_params = replicate(vqgan_params)

NameError: name 'params' is not defined